# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [126]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# 忽略警告訊息
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']                            # 取出目標值
df = df.drop(['PassengerId', 'Survived'] , axis=1)  # 刪去[乘客ID]、[目標值]欄位
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [127]:
df.nunique()            # 查看每一欄資料類別數

Pclass        3
Name        891
Sex           2
Age          88
SibSp         7
Parch         7
Ticket      681
Fare        248
Cabin       147
Embarked      3
dtype: int64

In [128]:

# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
# 這邊先取['Sex'] & ['Fare']
df['Sex'] = df['Sex'].fillna('None')                                              # 填補空缺直

# 對 ['Sex'] 進行分類，對每類別的 ['Fare'] 計算[指定值]
df_mean = df.groupby(['Sex'])['Fare'].mean().reset_index()                           # 計算[平均值]  
df_mode = df.groupby(['Sex'])['Fare'].apply( lambda x: x.mode()[0] ).reset_index()   # 計算[眾數]
df_median = df.groupby(['Sex'])['Fare'].median().reset_index()                       # 計算[中位數]
df_max = df.groupby(['Sex'])['Fare'].max().reset_index()                             # 計算[最大值]
df_min = df.groupby(['Sex'])['Fare'].min().reset_index()                             # 計算[最小值]
df_count = df.groupby(['Sex'])['Fare'].count().reset_index()                         # 計算[資料數量]
# 最後都會加上 .reset_index() 是為了要保留 ['Sex'] 欄位

# 合併資料
df_temp = pd.merge(df_mean, df_mode, on=['Sex'], how='left')
df_temp = pd.merge(df_temp, df_median, on=['Sex'], how='left')
df_temp = pd.merge(df_temp, df_max, on=['Sex'], how='left')
df_temp = pd.merge(df_temp, df_min, on=['Sex'], how='left')
df_temp = pd.merge(df_temp, df_count, on=['Sex'], how='left')

# 重新命名欄位
df_temp.columns = ['Sex','Fare_mean','Fare_mode','Fare_median','Fare_max','Fare_min','Fare_count']
df_temp


,Sex,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,female,44.479818,7.75,23.0,512.3292,6.75,314
1,male,25.523893,8.05,10.5,512.3292,0.00,577


In [129]:
# 合併新特徵與原始資料
df = pd.merge(df, df_temp, on=['Sex'], how='left')
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,25.523893,8.05,10.5,512.3292,0.00,577
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,44.479818,7.75,23.0,512.3292,6.75,314
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,44.479818,7.75,23.0,512.3292,6.75,314
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44.479818,7.75,23.0,512.3292,6.75,314
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,25.523893,8.05,10.5,512.3292,0.00,577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,25.523893,8.05,10.5,512.3292,0.00,577
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,44.479818,7.75,23.0,512.3292,6.75,314
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,44.479818,7.75,23.0,512.3292,6.75,314
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,25.523893,8.05,10.5,512.3292,0.00,577


In [130]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):   
    if dtype == 'float64' or dtype == 'int64':                      # 篩選出[數值型欄位]
        num_features.append(feature)                                # 將欄位名稱加入 list
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]                                               # 只留下數值型欄位
df = df.fillna(-1)                                                  # 填補空缺值
MMEncoder = MinMaxScaler()                                          # 設定最大最小化
df.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,3,22.0,1,0,7.2500,25.523893,8.05,10.5,512.3292,0.00,577
1,1,38.0,1,0,71.2833,44.479818,7.75,23.0,512.3292,6.75,314
2,3,26.0,0,0,7.9250,44.479818,7.75,23.0,512.3292,6.75,314
3,1,35.0,1,0,53.1000,44.479818,7.75,23.0,512.3292,6.75,314
4,3,35.0,0,0,8.0500,25.523893,8.05,10.5,512.3292,0.00,577


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

## 根據結果，分數有蠻明顯的提升，但我之後有測試用[Pclass]分類和[Fare]的關係，但分數卻反而變低，所以可能還是要依照其專業判斷選擇分類特徵

In [131]:
# 原始特徵 + 邏輯斯迴歸
df_orgin = df.drop(['Fare_mean','Fare_mode','Fare_median','Fare_max','Fare_min','Fare_count'], axis=1)          # 去掉新特徵，得到原始特徵
train_X = MMEncoder.fit_transform(df_orgin)                         # 最大最小化
estimator = LogisticRegression()                                    # 設定羅吉斯回歸
cross_val_score(estimator, train_X, train_Y, cv=5).mean()           # 計算分數

0.70041428661101

In [132]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7833720419308268